# Black-Scholes Option Pricer - Usage Examples

This notebook demonstrates how to use the `cc_optimizer.options` module for pricing European vanilla options and calculating Greeks.

## Table of Contents
1. [Basic Setup](#setup)
2. [Simple Pricing - Functional API](#functional-api)
3. [Advanced Usage - Class-Based API](#class-api)
4. [Calculating Greeks](#greeks)
5. [Crypto Option Examples](#crypto-examples)
6. [Integration with hist_vol_model](#iv-integration)
7. [Comparative Analysis](#analysis)

## 1. Basic Setup <a id="setup"></a>

First, let's import the necessary modules.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import option pricing modules
from cc_optimizer.options import (
    BlackScholesModel,
    Greeks,
    bs_pricer,
    calculate_greeks
)

# Display settings
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Modules imported successfully")

## 2. Simple Pricing - Functional API <a id="functional-api"></a>

The simplest way to price an option is using the functional API.

In [ ]:
# Price a simple ATM call option
call_price = bs_pricer(
    S=100,           # Spot price
    K=100,           # Strike price
    T=1,             # Time to expiry (1 year)
    r=0.05,          # Risk-free rate (5%)
    sigma=0.2,       # Volatility (20%)
    option_type='call'
)

print(f"ATM Call Option Price: ${call_price:.4f}")

In [ ]:
# Price a put option with the same parameters
put_price = bs_pricer(
    S=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    option_type='put'
)

print(f"ATM Put Option Price: ${put_price:.4f}")

# Verify put-call parity: C - P = S - K*e^(-rT)
parity_lhs = call_price - put_price
parity_rhs = 100 - 100 * np.exp(-0.05 * 1)

print(f"\nPut-Call Parity Check:")
print(f"  C - P = {parity_lhs:.6f}")
print(f"  S - K*e^(-rT) = {parity_rhs:.6f}")
print(f"  Difference: {abs(parity_lhs - parity_rhs):.10f}")

## 3. Advanced Usage - Class-Based API <a id="class-api"></a>

For more complex scenarios, use the class-based API which provides additional functionality.

In [ ]:
# Create a model instance
model = BlackScholesModel(
    S=110,           # Spot = 110 (ITM for call)
    K=100,
    T=0.5,           # 6 months
    r=0.05,
    sigma=0.25,
    option_type='call'
)

# Price the option
price = model.price()
intrinsic = model.intrinsic_value()
time_value = model.time_value()
moneyness = model.moneyness()

print(f"Option Details:")
print(f"  Price: ${price:.4f}")
print(f"  Intrinsic Value: ${intrinsic:.4f}")
print(f"  Time Value: ${time_value:.4f}")
print(f"  Moneyness: {moneyness}")

# Display full model info
print(f"\n{model}")

## 4. Calculating Greeks <a id="greeks"></a>

Greeks measure the sensitivities of the option price to various parameters.

In [ ]:
# Using the class-based API
model = BlackScholesModel(
    S=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    option_type='call'
)

greeks_calc = Greeks(model)

# Calculate individual Greeks
delta = greeks_calc.delta()
gamma = greeks_calc.gamma()
theta = greeks_calc.theta(per_day=True)  # Per day
vega = greeks_calc.vega(per_percent=True)  # Per 1% vol change

print("Greeks (Individual):")
print(f"  Delta: {delta:.4f}")
print(f"  Gamma: {gamma:.6f}")
print(f"  Theta: ${theta:.4f} per day")
print(f"  Vega:  ${vega:.4f} per 1% vol")

In [ ]:
# Calculate all Greeks at once (more efficient)
all_greeks = greeks_calc.all_greeks()

print("\nGreeks (All at once):")
for name, value in all_greeks.items():
    print(f"  {name.capitalize()}: {value:.6f}")

# Display formatted Greeks
print(f"\n{greeks_calc}")

In [ ]:
# Using the functional API
greeks_dict = calculate_greeks(
    S=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    option_type='put'
)

print("Put Option Greeks (Functional API):")
for name, value in greeks_dict.items():
    print(f"  {name.capitalize()}: {value:.6f}")

## 5. Crypto Option Examples <a id="crypto-examples"></a>

Examples with cryptocurrency-like parameters (high volatility, zero risk-free rate).

In [ ]:
# Bitcoin-like option (high vol, short-dated)
btc_call = BlackScholesModel(
    S=50000,         # BTC spot = $50,000
    K=50000,         # ATM
    T=7/365,         # 1 week
    r=0.0,           # No risk-free rate for crypto
    sigma=0.8,       # 80% annual volatility
    option_type='call'
)

print("BTC 1-Week ATM Call:")
print(f"  Price: ${btc_call.price():.2f}")
print(f"  % of Spot: {btc_call.price() / btc_call.S * 100:.2f}%")

# Calculate Greeks
btc_greeks = Greeks(btc_call)
print(f"\n  Delta: {btc_greeks.delta():.4f}")
print(f"  Gamma: {btc_greeks.gamma():.8f}")
print(f"  Theta: ${btc_greeks.theta():.2f} per day")
print(f"  Vega:  ${btc_greeks.vega():.2f} per 1% vol")

In [ ]:
# Ethereum-like option
eth_put = BlackScholesModel(
    S=3000,          # ETH spot = $3,000
    K=2800,          # 93% strike (protective put)
    T=30/365,        # 30 days
    r=0.0,
    sigma=1.0,       # 100% annual volatility
    option_type='put'
)

print("ETH 30-Day Protective Put (93% strike):")
print(f"  Price: ${eth_put.price():.2f}")
print(f"  Hedge Cost: {eth_put.price() / eth_put.S * 100:.2f}% of holdings")
print(f"  Moneyness: {eth_put.moneyness()}")

### Volatility Surface Analysis

In [ ]:
# Create a grid of strikes and time to expiries
spot = 100
strikes = np.linspace(80, 120, 9)
times = np.array([7, 14, 30, 60, 90]) / 365  # Days to years

# Calculate option prices for different strikes and times
results = []

for T in times:
    for K in strikes:
        model = BlackScholesModel(S=spot, K=K, T=T, r=0.0, sigma=0.6, option_type='call')
        greeks_calc = Greeks(model)
        
        results.append({
            'Strike': K,
            'Days': int(T * 365),
            'Moneyness': K / spot,
            'Price': model.price(),
            'Delta': greeks_calc.delta(),
            'Gamma': greeks_calc.gamma(),
            'Theta': greeks_calc.theta(),
            'Vega': greeks_calc.vega()
        })

df = pd.DataFrame(results)

print("\nOption Prices Across Strikes and Tenors:")
pivot = df.pivot(index='Strike', columns='Days', values='Price')
print(pivot)

In [ ]:
# Visualize option price vs strike for different tenors
fig, ax = plt.subplots(figsize=(10, 6))

for days in [7, 14, 30, 60, 90]:
    subset = df[df['Days'] == days]
    ax.plot(subset['Strike'], subset['Price'], marker='o', label=f'{days} days')

ax.axvline(spot, color='black', linestyle='--', alpha=0.3, label='Spot')
ax.set_xlabel('Strike Price', fontsize=12)
ax.set_ylabel('Call Option Price', fontsize=12)
ax.set_title('Option Price vs Strike (60% Vol)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Price visualization complete")

### Greeks Visualization

In [ ]:
# Visualize Greeks for 30-day options across strikes
subset_30d = df[df['Days'] == 30]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Delta
axes[0, 0].plot(subset_30d['Strike'], subset_30d['Delta'], 'b-', marker='o', linewidth=2)
axes[0, 0].axvline(spot, color='black', linestyle='--', alpha=0.3)
axes[0, 0].axhline(0.5, color='gray', linestyle=':', alpha=0.3)
axes[0, 0].set_xlabel('Strike Price')
axes[0, 0].set_ylabel('Delta')
axes[0, 0].set_title('Delta vs Strike (30 days)', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Gamma
axes[0, 1].plot(subset_30d['Strike'], subset_30d['Gamma'], 'g-', marker='o', linewidth=2)
axes[0, 1].axvline(spot, color='black', linestyle='--', alpha=0.3)
axes[0, 1].set_xlabel('Strike Price')
axes[0, 1].set_ylabel('Gamma')
axes[0, 1].set_title('Gamma vs Strike (30 days)', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Theta
axes[1, 0].plot(subset_30d['Strike'], subset_30d['Theta'], 'r-', marker='o', linewidth=2)
axes[1, 0].axvline(spot, color='black', linestyle='--', alpha=0.3)
axes[1, 0].axhline(0, color='gray', linestyle=':', alpha=0.3)
axes[1, 0].set_xlabel('Strike Price')
axes[1, 0].set_ylabel('Theta (per day)')
axes[1, 0].set_title('Theta vs Strike (30 days)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Vega
axes[1, 1].plot(subset_30d['Strike'], subset_30d['Vega'], 'm-', marker='o', linewidth=2)
axes[1, 1].axvline(spot, color='black', linestyle='--', alpha=0.3)
axes[1, 1].set_xlabel('Strike Price')
axes[1, 1].set_ylabel('Vega (per 1%)')
axes[1, 1].set_title('Vega vs Strike (30 days)', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Greeks visualization complete")

## 6. Integration with hist_vol_model <a id="iv-integration"></a>

Use implied volatility from the `hist_vol_model` project (if available).

In [ ]:
# Example: Create model using IV from hist_vol_model
# NOTE: This requires the hist_vol_model project to be in the expected location

try:
    # Create model with IV from hist_vol_model
    model_with_iv = BlackScholesModel.from_iv_model(
        S=50000,              # Current BTC price
        K=50000,              # ATM strike
        T=30/365,             # 30 days
        r=0.0,                # No risk-free rate
        token='BTC',          # Token symbol
        tenor_days=30,        # 30-day tenor
        strike_percent=100.0, # ATM (100% of spot)
        option_type='call',
        iv_model='model_01',  # IV model to use
        quote_type='mid'      # Use mid quote
    )
    
    print("Model created with IV from hist_vol_model:")
    print(f"  Implied Vol: {model_with_iv.sigma:.2%}")
    print(f"  Option Price: ${model_with_iv.price():.2f}")
    
except ImportError as e:
    print(f"Could not import hist_vol_model: {e}")
    print("\nThis is expected if hist_vol_model is not set up.")
    print("Use manual sigma parameter instead:")
    
    model_manual = BlackScholesModel(
        S=50000,
        K=50000,
        T=30/365,
        r=0.0,
        sigma=0.75,  # Manually specified 75% vol
        option_type='call'
    )
    
    print(f"  Manual Vol: {model_manual.sigma:.2%}")
    print(f"  Option Price: ${model_manual.price():.2f}")

## 7. Comparative Analysis <a id="analysis"></a>

Compare options with different parameters.

In [ ]:
# Compare ATM options with different volatilities
vols = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
comparison = []

for vol in vols:
    call_model = BlackScholesModel(S=100, K=100, T=30/365, r=0.0, sigma=vol, option_type='call')
    put_model = BlackScholesModel(S=100, K=100, T=30/365, r=0.0, sigma=vol, option_type='put')
    
    call_greeks = Greeks(call_model)
    put_greeks = Greeks(put_model)
    
    comparison.append({
        'Volatility': f"{vol:.0%}",
        'Call Price': call_model.price(),
        'Put Price': put_model.price(),
        'Call Delta': call_greeks.delta(),
        'Put Delta': put_greeks.delta(),
        'Gamma': call_greeks.gamma(),
        'Vega': call_greeks.vega()
    })

comparison_df = pd.DataFrame(comparison)
print("\nATM 30-Day Option Comparison (S=K=100, r=0):")
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize impact of volatility on option price
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price vs Volatility
ax1.plot(vols, comparison_df['Call Price'], 'b-', marker='o', linewidth=2, label='Call')
ax1.plot(vols, comparison_df['Put Price'], 'r-', marker='s', linewidth=2, label='Put')
ax1.set_xlabel('Volatility (annual)', fontsize=12)
ax1.set_ylabel('Option Price', fontsize=12)
ax1.set_title('Option Price vs Volatility\n(ATM, 30 days)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Vega vs Volatility
ax2.plot(vols, comparison_df['Vega'], 'g-', marker='o', linewidth=2)
ax2.set_xlabel('Volatility (annual)', fontsize=12)
ax2.set_ylabel('Vega (per 1%)', fontsize=12)
ax2.set_title('Vega vs Volatility\n(ATM, 30 days)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Volatility comparison complete")

## Summary

This notebook demonstrated:

1. **Functional API**: Quick pricing with `bs_pricer()` and `calculate_greeks()`
2. **Class-based API**: Advanced features with `BlackScholesModel` and `Greeks` classes
3. **Greeks Calculation**: Delta, Gamma, Theta, Vega for risk management
4. **Crypto Examples**: High volatility, short-dated options typical of crypto markets
5. **Visualization**: Price and Greeks behavior across strikes and tenors
6. **IV Integration**: Optional integration with `hist_vol_model` project
7. **Comparative Analysis**: Impact of volatility on option pricing

### Key Takeaways

- **Delta**: Hedge ratio, ranges from 0 to 1 (calls) or -1 to 0 (puts)
- **Gamma**: Rate of delta change, highest at ATM
- **Theta**: Time decay, typically negative for long options
- **Vega**: Volatility sensitivity, higher for longer-dated options

### Next Steps

- Integrate with real market data (Deribit, exchange APIs)
- Implement implied volatility solver (reverse problem)
- Compare BS prices with actual market prices
- Explore advanced models (jump-diffusion, stochastic vol)